In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

from collections import Counter,defaultdict
from itertools import chain, combinations

import warnings
warnings.filterwarnings('ignore')

import imshow
from wordcloud import WordCloud
from PIL import Image

from itertools import chain
from collections import Counter
import pickle

import re
import sent2vec
import hnswlib
from gensim.models import Word2Vec
from gensim import corpora
from scipy.sparse import *
from scipy.sparse.linalg import svds

import math

from py_file.total_processing import konlpy_preprocessing
kp = konlpy_preprocessing()

SyntaxError: unexpected EOF while parsing (total_processing.py, line 44)

In [11]:
song_meta = pd.read_json('data/song_meta.json')
song_meta['artists'] = song_meta['artist_name_basket'].apply(lambda x : " ".join(x))
train = pd.read_json('data/train.json')

display(song_meta.head(1))
display(train.head(1))

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,artists
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,Various Artists


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000


# 모델 불러오기

이전 파일 __3_1.Word2Vec_Knn_model_make.ipynb에서 생성한 word2vec와 knn모델들을 불러온다

In [35]:
s2v_model = sent2vec.Sent2vecModel()
s2v_model.load_model('/home/ubuntu/workspace/jh/project/model/s2v_model.bin')

p_title = pickle.load(open('/home/ubuntu/workspace/jh/project/model/p_title.pickle', 'rb'))
p_tags = pickle.load(open('/home/ubuntu/workspace/jh/project/model/p_tags.pickle', 'rb'))
p_new_tags = pickle.load(open('/home/ubuntu/workspace/jh/project/model/p_new_tags.pickle', 'rb'))

# 1. sent2vec를 통해 연관 플레이리스트 뽑아오기

이전 파일인 __4_1.Word2Vec_recommend.ipynb 와 같은 방식으로 진행됨

In [36]:
def s2v_recommendation(text, k_n = 2):
    test_tag = kp.konlpy_preprocessing(text)
    print(test_tag)
    test_tag = test_tag.split()
    test_tag = " ".join([i for i in test_tag if s2v_model.embed_sentence(i).sum() != 0])

    if s2v_model.embed_sentence(test_tag).sum() == 0.0:
        print('검색 결과를 찾을 수 없습니다!')

    else:
        emb = s2v_model.embed_sentence(test_tag)
        title_labels, title_distances = p_title.knn_query(emb, k = k_n, num_threads=8)
        tag_labels, tags_distances = p_tags.knn_query(emb, k = k_n, num_threads=8)
        new_tag_labels, new_tags_distances = p_new_tags.knn_query(emb, k = k_n, num_threads=8)

        reco_idx = list(chain.from_iterable(zip(title_labels.reshape(-1),tag_labels.reshape(-1),new_tag_labels.reshape(-1))))
        
        song_idx = list(set(np.concatenate(train.iloc[reco_idx]['songs'].tolist())))

        reco_songs = song_meta.iloc[song_idx,:][['song_name','artists']]
        
        return reco_songs.sample(30)

In [41]:
s2v_recommendation('여자아이들')

여자 아이들


,song_name,artists
251201,닥쳐줘요 (Feat. 솔지 Of EXID),배치기
205726,빙글뱅글 (Bingle Bangle),AOA
465654,주인공,선미
406356,질투 (Duet. 키썸),유성은
435750,Ride,나아람
632977,SOLO,제니 (JENNIE)
265258,내 집에서 나가 (Feat. 권정열 Of 10cm),현아
297411,가시나,선미
254540,권태기 - 연인송 (Feat. BESTie 혜연),유승우
124663,SIGNAL,TWICE (트와이스)
